In [1]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

'This script goes along the blog post\n"Building powerful image classification models using very little data"\nfrom blog.keras.io.\nIt uses data that can be downloaded at:\nhttps://www.kaggle.com/c/dogs-vs-cats/data\nIn our setup, we:\n- created a data/ folder\n- created train/ and validation/ subfolders inside data/\n- created cats/ and dogs/ subfolders inside train/ and validation/\n- put the cat pictures index 0-999 in data/train/cats\n- put the cat pictures index 1000-1400 in data/validation/cats\n- put the dogs pictures index 12500-13499 in data/train/dogs\n- put the dog pictures index 13500-13900 in data/validation/dogs\nSo that we have 1000 training examples for each class, and 400 validation examples for each class.\nIn summary, this is our directory structure:\n```\ndata/\n    train/\n        dogs/\n            dog001.jpg\n            dog002.jpg\n            ...\n        cats/\n            cat001.jpg\n            cat002.jpg\n            ...\n    validation/\n        dogs/\n   

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K



In [9]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/content/drive/MyDrive/CommonFiles/MUSA650-Data/catsdogs/train'
validation_data_dir = '/content/drive/MyDrive/CommonFiles/MUSA650-Data/catsdogs/validation'
test_data_dir = '/content/drive/MyDrive/CommonFiles/MUSA650-Data/catsdogs/test'
nb_train_samples = 20
nb_validation_samples = 18
nb_test_samples = 10
epochs = 5
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [12]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)



In [13]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 20 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [15]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('/content/drive/MyDrive/CommonFiles/MUSA650-Data/catsdogs/model_trained_v1.h5')

<ipython-input-15-083ce8f208fe>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
2/2 [==============================] - 2s 882ms/step - loss: 0.6871 - accuracy: 0.6000 - val_loss: 0.6963 - val_accuracy: 0.4000
Epoch 2/5
2/2 [==============================] - 2s 775ms/step - loss: 0.6625 - accuracy: 0.6000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/5
2/2 [==============================] - 1s 456ms/step - loss: 0.7199 - accuracy: 0.5000 - val_loss: 0.6951 - val_accuracy: 0.4000
Epoch 4/5
2/2 [==============================] - 1s 418ms/step - loss: 0.6992 - accuracy: 0.5000 - val_loss: 0.7277 - val_accuracy: 0.4000
Epoch 5/5
2/2 [==============================] - 1s 576ms/step - loss: 0.6749 - accuracy: 0.5500 - val_loss: 0.6989 - val_accuracy: 0.4000


In [16]:
model.evaluate(validation_generator)

2/2 [==============================] - 0s 94ms/step - loss: 0.7001 - accuracy: 0.3889


[0.7000697255134583, 0.3888888955116272]

In [17]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 10 images belonging to 2 classes.


In [18]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')

1/1 [==============================] - 0s 223ms/step - loss: 0.6758 - accuracy: 0.6000
Test accuracy: 0.6000000238418579
